# Grove I2C and GPIO Libraries

To make interacting with Grove peripherals easier there are two Libraries that can be used: `ipython_microblaze.I2C` and `ipython_microblaze.GPIO`. These libraries provide a uniform API regardless of what connection is used. Please note, there are some limitations on what peripherals can be connected to the various connectors. For the PYNQ Arduino shield only the dedicated I2C connectors can be used for I2C and any of the G1 to G7 connections used for GPIO. For the PYNQ PMOD-Grove adapter, all of the ports can be used for GPIO but on G3 and G4 can be used for I2C.

To start with we'll load the `ipython_microblaze` library and have a look at the API for the two devices.

In [1]:
import ipython_microblaze as ipmb

In [2]:
print(ipmb.I2C.declaration)


void i2c_connect(unsigned char port);
void i2c_read(unsigned char address, unsigned char* data, int length);
void i2c_write(unsigned char address, unsigned char* data, int length);



In [3]:
print(ipmb.GPIO.declaration)



#define GPIO_IN 1
#define GPIO_OUT 0

typedef int gpio;

gpio gpio_connect(unsigned char port, unsigned char wire);
void gpio_write(gpio channel, unsigned char value);
unsigned char gpio_read(gpio channel);
void gpio_set_direction(gpio channel, unsigned char direction);




## Using the GPIO

The Grove connector has two GPIO pins which can be accessed using channels 0 and 1 in the library. First we need to load the overlay and then we can read the value of the two pins back to python. The GPIO library requires creating instances for each pin so two instances are required to access both data pins in a Grove connector.

In [4]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay('base.bit')

In [5]:
%%microblaze base.PMODA gpio_read ipmb.GPIO
#include <unistd.h>

int main() {
    unsigned char data[2];
    gpio g1a = gpio_connect(G1, 0);
    gpio g1b = gpio_connect(G1, 1);
    gpio_set_direction(g1a, GPIO_IN);
    gpio_set_direction(g1b, GPIO_IN);
    data[0] = gpio_read(g1a);
    data[1] = gpio_read(g1b);
    write(STDOUT_FILENO, data, 2);
}

In [6]:
gpio_read.stream.read()

b'\x01\x00'

Likewise the same can be done for an Arduino-attached Grove device

In [7]:
%%microblaze base.ARDUINO gpio_read_a ipmb.GPIO
#include <unistd.h>

int main() {
    unsigned char data[2];
    gpio g1a = gpio_connect(G1, 0);
    gpio g1b = gpio_connect(G1, 1);
    gpio_set_direction(g1a, GPIO_IN);
    gpio_set_direction(g1b, GPIO_IN);
    data[0] = gpio_read(g1a);
    data[1] = gpio_read(g1b);
    write(STDOUT_FILENO, data, 2);
}

In [8]:
gpio_read_a.stream.read()

b'\x01\x00'

In [9]:
gpio_read.reset()
gpio_read_a.reset()

## Using I2C

The I2C peripherals can be written and read from as per the data sheet. This example communicates the Grove ADC and returns the raw value back to python

In [11]:
%%microblaze base.PMODA adc_read ipmb.I2C
#include <unistd.h>
#define ADDRESS 0x50

int main() {
    i2c_connect(G4);
    unsigned char data[2];
    // Setup the device
    data[0] = 0x02;
    data[1] = 0x20;
    i2c_write(ADDRESS, data, 2);
    // Read the value
    data[0] = 0x00;
    i2c_write(ADDRESS, data, 1);
    i2c_read(ADDRESS, data, 2);
    write(STDOUT_FILENO, data, 2);
}

In [12]:
adc_read.stream.read()

b'\x041'

Likewise we can do the same for Arduino

In [13]:
%%microblaze base.ARDUINO adc_read_a ipmb.I2C
#include <unistd.h>
#define ADDRESS 0x50

int main() {
    i2c_connect(G4);
    unsigned char data[2];
    // Setup the device
    data[0] = 0x02;
    data[1] = 0x20;
    i2c_write(ADDRESS, data, 2);
    // Read the value
    data[0] = 0x00;
    i2c_write(ADDRESS, data, 1);
    i2c_read(ADDRESS, data, 2);
    write(STDOUT_FILENO, data, 2);
}

In [14]:
adc_read_a.stream.read()

b'\x042'

Note that this is a common pattern for accessing registers in I2C devices - a single byte address write followed by a read or write of one or more bytes.